# Low-frequency Processing of DAS Data

This Jupyter Notebook is created for low-frequency processing of a [spool](https://dascore.org/tutorial/concepts.html#:~:text=read%20the%20docs!-,Data%20structures,-DASCore%20has%20two) of distributed acoustic sensing (DAS) data. It uses [DASCore](https://dascore.org/) package and the ```lf_das.py``` script and it is inspired by Dr. Ge Jin's [low frequency processing](https://github.com/DASDAE/DASLowFreqProcessing). 


<svg width="100%" height="1">
  <line x1="0" y1="0" x2="100%" y2="0" style="stroke:rgb(0,0,0);stroke-width:2" />
</svg>


#### Notes: 
1. Before using this notebook, make sure you have included the ```lf_das.py``` script in the current directory with this notebook and successfully installed DASCore using ```pip``` or ```conda```:
    ```python
    pip install dascore
    ```
    or
    ```python
    conda install dascore -c conda-forge
    ```   
2. Please find all supported I/O [here](https://dascore.quarto.pub/dascore/).
 

Current DASCore version: 0.0.13 (tested)

Date: 09/07/2023


Contact: [Ahmad Tourei](https://github.com/ahmadtourei/)

ahmadtourei@gmail.com

In [ ]:
# import libraries
import warnings
warnings.simplefilter('ignore')

import dascore as dc
import matplotlib.pyplot as plt
import numpy as np
import scipy

from time import time
from lf_das import LFProc, get_edge_effect_time, get_patch_time, waterfall_plot


### Get a spool of data to work on

In [ ]:
# define data path (spool of data) and output folder 
data_path = '/mnt/h/a'
output_data_folder =  '/mnt/h/results'
output_figure_folder = '/mnt/h/figures'

# get the sorted spool of data form the defined data path (on first run, it will index the patches and subsequently update the index file for future uses)
sp = dc.spool(data_path).sort("time").update()

# print the contents of first 5 patches
content_df = sp.get_contents()
content_df.head()

### Get some metadata and define a sub spool (if needed)

In [ ]:
# get sampling rate, channel spacing, and gauge length from the first patch
patch_0 = sp[0]
gauge_length = patch_0.attrs['gauge_length']
print("Gauge length = ", gauge_length)
channel_spacing = patch_0.attrs['distance_step']
print("Channel spacing = ", channel_spacing)
sampling_interval = patch_0.attrs['time_step']
print("Sampling interval = ", sampling_interval)
sampling_rate = 1/(sampling_interval / np.timedelta64(1, 's'))
print("Sampling rate = ", sampling_rate)

# select a sub-spool
t_1 = '2023-03-22 03:00:00'
t_2 = '2023-03-22 07:00:00'
ch_start = 400
ch_end = 1400
d_1 = patch_0.coords['distance'][ch_start] # in meter
d_2 = patch_0.coords['distance'][ch_end] # in meter
# or:
# d_1 = -115 # in meter
# d_2 = 2000 # in meter
sub_sp = sp.select(distance=(d_1, d_2), time=(t_1, t_2)) 


### Get low-pass filter parameters

In [ ]:
# define memory size that you'd like to dedicate to low-frequency processing 
memory_size = 10000 # in MB
patch_length = get_patch_time(memory_size=memory_size, sampling_rate=sampling_rate, num_ch=ch_end-ch_start)
print('patch_length = ', patch_length, str(' sec.'))

# define the target sampling interval in seconds
d_t = 10.0 # so, cutoff_freq = Nyq_new = 1/(2*d_t) = 0.05 hz

# define the desired tolerance for getting the edge time (smaller tolerance results a longer eliminated edges in each patch and higher accuracy. 1e-3 is recommended.)
tolerance = 1e-3
edge_buffer = get_edge_effect_time(sampling_interval=1/sampling_rate, total_T=patch_length, tol=tolerance, freq=1/d_t)
print('edge_buffer = ', edge_buffer, str(' sec.'))

# pass the spool to the LFProc class
lfp = LFProc(sub_sp)
lfp.update_processing_parameter(output_sample_interval=d_t, process_patch_size=int(patch_length/d_t), edge_buff_size=int(np.ceil(edge_buffer/d_t)))

# set the output folder - Caution: If you set delete_existing=True, you will remove all contents in the output_data_folder directory. If you set delete_existing=False, you need to have a empty output_data_folder directory to proceed.
lfp.set_output_folder(output_data_folder, delete_existing=False)


### Do low-frequency processing a portion of the spool

In [ ]:
tic = time()
# do lowpass processing on (t_1,t_2) time range
t_1 = np.datetime64('2023-03-22T03:00:00')
t_2 = np.datetime64('2023-03-22T07:00:00')
lfp.process_time_range(t_1,t_2)
toc = time()
print(f'processing time (sec): {toc-tic}')


### Merge the results before visualization

In [ ]:
sp_result = dc.spool(output_data_folder)
sp_result = sp_result.chunk(time=None) 


### Visualize the low-pass filtered results

#### a) Seismogram using Matplotlib 

In [ ]:
# define the channel of interest
channel = 1330
ch_inx = channel - ch_start
filtered_data = sp_result[0].data[:, ch_inx]
n_samples = filtered_data.shape[0]
num_sec = int(n_samples*d_t)
t_filt = np.linspace(0, num_sec, n_samples, endpoint=False)

# define the scale to apply to the raw data
scale_iDAS = float((116*sampling_rate/gauge_length)/1e9) # to strain rate

# define the channel range whose mean value you want to subtract from the DAS trace
ch_start_demean = 400
ch_end_demean = 600
demeaned_scaled = (filtered_data - np.mean(sp_result[0].data[:,ch_start_demean-ch_start:ch_end_demean-ch_end], axis=1)) * scale_iDAS

# plot the result
plt.figure(figsize=(12,8))
plt.plot(t_filt, demeaned_scaled, label='Low-freq. Silixa - channel: ' + str(channel))
plt.legend(loc='best')
plt.ylabel('Strain rate (1/sec)')
plt.xlabel('Time (sec) \n (4 hours, starting from 2023/03/22 03:00:00 UTC)')
plt.title('Filtered signal using DASDAE - Stage 8')
plt.grid(True)

# save the figure into the output_figure_folder directory
file_name_lowfreq = '/dascore_lowpass_' + str(int(d_t*2)) + 'sec_filter_channel' + str(channel) + '.jpeg'
plt.savefig(output_figure_folder + file_name_lowfreq, dpi=600, format='jpeg')
plt.show()


##### Apply a median filter to remove noise and microseismic events


In [ ]:
# define window length (number of samples) for the median filter
window_size = 9
median_filtered_signal = scipy.ndimage.median_filter(demeaned_scaled, size=window_size)       

plt.figure(figsize=(12,8))
plt.plot(t_filt, median_filtered_signal, label='Low-freq. Silixa - channel: ' + str(channel))
plt.legend(loc='best')
plt.ylabel('Strain rate (1/sec)')
plt.xlabel('Time (sec) \n (4 hours, starting from 2023/03/22 03:00:00 UTC)')
# plt.ylim(-6e-6, 6e-6)
plt.title('Filtered signal using DASDAE - Stage 8')
plt.grid(True)

# save the figure into the output_figure_folder directory
file_name_lowfreq = '/dascore_lowpass_' + str(int(d_t*2)) + 'sec_filtered_median_filtered_channel' + str(channel) + '.jpeg'
plt.savefig(output_figure_folder + file_name_lowfreq, dpi=600, format='jpeg')
plt.show()


#### b) Waterfall plot using Matplotlib 

In [ ]:
# define the scale to apply to the raw data
scale_iDAS = float((116*sampling_rate/gauge_length)/1e9) # to strain rate

# get the filtered results from the result spool
filtered_data = sp_result[0].data

# define the channel range whose mean you want to subtract from the DAS trace
ch_start_demean = 400
ch_end_demean = 600
mean_array = np.mean(filtered_data[:,ch_start_demean-ch_start:ch_end_demean-ch_start], axis=1)  # Calculate the mean along axis 1
mean_array = mean_array.reshape(-1, 1)  # Reshape mean_array to match the shape of full_array

# demean and scale the results
demeaned_scaled_data = (filtered_data - mean_array) * scale_iDAS

# define the channles and seconds you want to plot
min_sec = 0
max_sec = 13990
min_ch = 400
max_ch = 1355
fig_title = "Silixa's low-freq. DAS - Stage 8"
waterfall_plot(demeaned_scaled_data.T, min_sec, max_sec, min_ch-ch_start, max_ch-ch_start, 1185, 1/d_t, fig_title, output_figure_folder, "low_freq_raster_plot")


##### Apply a median filter to remove noise and microseismic events


In [ ]:
# define window length (number of samples) for the median filter
window_size = 5 # means 5*d_t seconds
median_filtered_signal = scipy.ndimage.median_filter(demeaned_scaled_data, size=window_size)  

# define the channles and seconds you want to plot
min_sec = 0
max_sec = 13990
min_ch = 400
max_ch = 1355
fig_title = "Silixa's low-freq. DAS - Median filtered - Stage 8"
waterfall_plot(median_filtered_signal.T, min_sec, max_sec, min_ch-ch_start, max_ch-ch_start,1185, 1/d_t, fig_title, output_figure_folder, "low_freq_raster_plot_median_filtered")


#### c) Waterfall plot using DASCore

In [ ]:
# define the scale to apply to the raw data
scale_iDAS = float((116*sampling_rate/gauge_length)/1e9) # to strain rate

# select a sub spool for visualization
min_ch = 400
max_ch = 1355
d_1 = sp_result[0].coords['distance'][min_ch-ch_start] # in meter
d_2 = sp_result[0].coords['distance'][max_ch-ch_start] # in meter
sub_sp_result = sp_result.select(distance=(d_1, d_2))

# get the filtered data from the saved result
sub_sp_result_merged = sub_sp_result.chunk(time=None)
filtered_data = sub_sp_result_merged[0].data

# define the channel range whose mean you want to subtract from the DAS trace
ch_start_demean = 400
ch_end_demean = 600
mean_array = np.mean(filtered_data[:,ch_start_demean-ch_start:ch_end_demean-ch_start], axis=1)  # Calculate the mean along axis 1
mean_array = mean_array.reshape(-1, 1)  # Reshape mean_array to match the shape of full_array

# demean and scale the results
demeaned_scaled_data = (filtered_data - mean_array) * scale_iDAS

# make a new patch containing the demeaned_scaled_data data
filtered_demeaned_scaled = sub_sp_result_merged[0].new(data=demeaned_scaled_data)

# plot
filtered_demeaned_scaled.viz.waterfall(scale=0.01) # scale acts as a clipper for data
